In [ ]:
import pandas as pd

# 1. Excel 파일 불러오기
df_weather = pd.read_excel("오이.xlsx")

# 2. 선택할 Column 목록 정의
selected_columns = ['TM', 'STN',                                                 # TM: 관측 날짜, STN: 관측 지점
                    'TA_AVG', 'TA_MAX', 'TA_MIN', 'TD_AVG', 'TS_AVG', 'TG_MIN',  # 기온 관련 변수
                    'HM_AVG', 'HM_MIN', 'EV_S', 'EV_L', 'PV_AVG',                # 습도 및 증발 관련 변수
                    'SS_DAY', 'SS_DUR', 'SI_DAY',                                # 일조 및 일사 관련 변수
                    'RN_DAY', 'RN_DUR', 'RN_60M_MAX', 'SD_NEW', 'SD_MAX',        # 강수 관련 변수
                    'WS_AVG', 'WS_MAX']                                          # 풍속 관련 변수

# 3. Column 선택
df_selected = df_weather[selected_columns].copy()

# 4. 새로운 csv 파일로 저장
df_selected.to_csv("오이_selected.csv", index = False, encoding = 'utf-8')

In [35]:
# Preprocessing

import numpy as np
import pandas as pd

# ===== 설정 =====
FILE_IN  = "오이_selected.csv"
FILE_OUT = "오이_preprocessed.csv"

START, END = "2017-01-01", "2024-12-31"
APPLY_WINSOR = True

PRICE_IMPUTE_MODE   = "past_only"
PRICE_SHORT_GAP_MAX = 3
PRICE_ROLL          = 7

CLIMATE_VARS = [
    "TA_AVG","TA_MAX","TA_MIN","TD_AVG","TS_AVG","TG_MIN",
    "HM_AVG","HM_MIN","EV_S","EV_L","PV_AVG",
    "SS_DAY","SS_DUR","SI_DAY",
    "RN_DAY","RN_DUR","RN_60M_MAX","SD_NEW","SD_MAX",
    "WS_AVG","WS_MAX",
]

SENTINELS = {-9, -9.0, -9.9, -99, -99.0, -99.9, -999, -999.0, -999.9, -9999, -9999.0}

# ===== 유틸 =====
def to_numeric_safely(s: pd.Series) -> pd.Series:
    if s.dtype.kind in "biufc": 
        return s.astype(float)
    s2 = s.astype(str).str.strip()
    s2 = s2.replace([r"^\s*$", r"^-+$", r"^nan$", r"^none$", r"^null$", r"^na$"], np.nan, regex=True)
    s2 = s2.str.replace(",", "", regex=False)
    s2 = s2.str.replace(r"[^0-9\.\-]", "", regex=True)
    return pd.to_numeric(s2, errors="coerce")

def replace_sentinels(df, cols, sentinels=SENTINELS):
    out = df.copy()
    for c in cols:
        if c in out.columns:
            out[c] = out[c].where(~out[c].isin(sentinels), np.nan)
    return out

def sanitize_ranges(df, cols):
    out = df.copy()
    for c in cols:
        if c.startswith("HM"):
            out.loc[(out[c] < 0) | (out[c] > 100), c] = np.nan
        if c.startswith(("TA_","TD_","TS_","TG_")):
            out.loc[(out[c] < -50) | (out[c] > 60), c] = np.nan
        if c.startswith(("WS_")):
            out.loc[out[c] < 0, c] = np.nan
        if c.startswith(("RN_","SD_","EV_")):
            out.loc[out[c] < 0, c] = np.nan
        if c == "RN_DUR":
            out.loc[(out[c] < 0) | (out[c] > 24), c] = np.nan
    return out

def run_lengths_na(s):
    isna = s.isna().astype(int)
    grp = (isna != isna.shift(1)).cumsum()
    return isna.groupby(grp).transform('sum') * isna

def fill_short_ffill(df, col, max_len):
    s = df[col].copy()
    rl = run_lengths_na(s)
    mask_short = (rl > 0) & (rl <= max_len)
    s[mask_short] = s.ffill()[mask_short]  # 과거만 사용
    df[col] = s
    return df

def doy_stat_past(series, dates, fn="median", cond=None, q=None):
    out = pd.Series(index=series.index, dtype=float)
    doy = dates.dt.dayofyear
    for i in range(len(series)):
        if pd.isna(series.iat[i]):
            d = doy.iat[i]
            mask = (dates < dates.iat[i]) & (doy == d)
            if cond is not None: mask &= cond
            past = series[mask].dropna()
            if len(past):
                out.iat[i] = past.median() if fn=="median" else past.quantile(q or 0.95)
            else:
                out.iat[i] = np.nan
    return out

def winsorize_by_doy_past(df, col, dates, low=0.01, high=0.99):
    s = df[col].copy(); doy = dates.dt.dayofyear; out = s.copy()
    for i in range(len(s)):
        past = s[(dates < dates.iat[i]) & (doy == doy.iat[i])].dropna()
        if len(past) >= 30:
            lo, hi = past.quantile(low), past.quantile(high)
            if out.iat[i] < lo: out.iat[i] = lo
            if out.iat[i] > hi: out.iat[i] = hi
    return out

# ===== 0) 로드 & 날짜 표준화 =====
df = pd.read_csv(FILE_IN)
# (헤더 안전) 공백/BOM 제거
df.columns = df.columns.str.replace("\ufeff","", regex=False).str.strip()

if "TM" not in df.columns:
    raise ValueError("CSV에 TM 컬럼(날짜)이 없습니다.")
df = df.rename(columns={"TM": "date"})
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# 타깃(price) = 마지막 열
price_col = df.columns[-1]
assert price_col != "date", "마지막 열이 날짜로 잡혔어요. 파일 구조 확인 필요."

# 숫자형 변환 + 센티널 → NaN
present_vars = [c for c in CLIMATE_VARS if c in df.columns]
for c in present_vars + [price_col]:
    if c in df.columns: 
        df[c] = to_numeric_safely(df[c])
df = replace_sentinels(df, present_vars, SENTINELS)

# ① 전체가 결측인 칼럼 먼저 드롭
all_na_init = [c for c in present_vars if df[c].isna().all()]
if all_na_init:
    print("드롭(원본 전체-NaN):", all_na_init)
    df = df.drop(columns=all_na_init)
    present_vars = [c for c in present_vars if c not in all_na_init]

# ===== 1) 날짜 정렬 & 중복일 평균 → 연속 캘린더 =====
df = df.sort_values("date")
df = df.groupby("date", as_index=False).mean(numeric_only=True)
full = pd.DataFrame({"date": pd.date_range(START, END, freq="D")})
df = full.merge(df, on="date", how="left")
df["year"] = df["date"].dt.year

# ===== 2) 물리범위 sanitize → NaN =====
df = sanitize_ranges(df, present_vars)

# ② 강수·적설 결측=0
for c in ["RN_DAY", "RN_60M_MAX", "RN_DUR", "SD_NEW", "SD_MAX"]:
    if c in df.columns:
        df[c] = df[c].fillna(0.0)

# 하한/상한 및 제약식
if "RN_DAY" in df.columns:
    df["RN_DAY"] = df["RN_DAY"].clip(lower=0)
if "RN_60M_MAX" in df.columns:
    df["RN_60M_MAX"] = df["RN_60M_MAX"].clip(lower=0)
if "RN_DUR" in df.columns:
    df["RN_DUR"] = df["RN_DUR"].clip(lower=0, upper=24)
if "SD_NEW" in df.columns:
    df["SD_NEW"] = df["SD_NEW"].clip(lower=0)
if "SD_MAX" in df.columns:
    df["SD_MAX"] = df["SD_MAX"].clip(lower=0)
if {"RN_DAY","RN_60M_MAX"}.issubset(df.columns):
    df["RN_60M_MAX"] = np.minimum(df["RN_60M_MAX"], df["RN_DAY"])
if {"SD_NEW","SD_MAX"}.issubset(df.columns):
    df["SD_MAX"] = np.maximum(df["SD_MAX"], df["SD_NEW"])

# ===== 3) 기후 변수군별 결측 보간 =====
# (A) 온도
temp_cols = [c for c in ["TA_AVG","TA_MAX","TA_MIN","TD_AVG","TS_AVG","TG_MIN"] if c in df.columns]
for c in temp_cols:
    df = fill_short_ffill(df, c, 3)
    miss = df[c].isna()
    if miss.any(): 
        df.loc[miss, c] = doy_stat_past(df[c], df["date"], "median")[miss]
if {"TA_MAX","TA_MIN"}.issubset(df.columns):
    daily_range = (df["TA_MAX"] - df["TA_MIN"]).rolling(60, min_periods=15).median()
    if "TA_AVG" in df.columns:
        m = df["TA_AVG"].isna() & df["TA_MAX"].notna() & df["TA_MIN"].notna()
        df.loc[m, "TA_AVG"] = (df.loc[m, "TA_MAX"] + df.loc[m, "TA_MIN"]) / 2
    m = ("TA_AVG" in df.columns) and df["TA_MAX"].isna() & df["TA_AVG"].notna() & df["TA_MIN"].notna()
    if isinstance(m, pd.Series) and m.any():
        df.loc[m, "TA_MAX"] = df.loc[m, "TA_AVG"] + (daily_range[m] / 2.0)
    m = ("TA_AVG" in df.columns) and df["TA_MIN"].isna() & df["TA_AVG"].notna() & df["TA_MAX"].notna()
    if isinstance(m, pd.Series) and m.any():
        df.loc[m, "TA_MIN"] = df.loc[m, "TA_AVG"] - (daily_range[m] / 2.0)
if {"TA_MIN","TA_AVG","TA_MAX"}.issubset(df.columns):
    df["TA_MIN"] = np.minimum(df["TA_MIN"], df["TA_AVG"])
    df["TA_MAX"] = np.maximum(df["TA_MAX"], df["TA_AVG"])

# (B) 습도
for c in [x for x in ["HM_AVG","HM_MIN"] if x in df.columns]:
    df = fill_short_ffill(df, c, 3)
    miss = df[c].isna()
    if miss.any(): 
        df.loc[miss, c] = doy_stat_past(df[c], df["date"], "median")[miss]
if {"HM_MIN","HM_AVG"}.issubset(df.columns):
    delta_med = (df["HM_AVG"] - df["HM_MIN"]).rolling(60, min_periods=15).median()
    m = df["HM_AVG"].isna() & df["HM_MIN"].notna()
    if isinstance(m, pd.Series) and m.any():
        df.loc[m, "HM_AVG"] = np.minimum(df.loc[m, "HM_MIN"] + delta_med[m], 100)
    df["HM_MIN"] = np.minimum(df["HM_MIN"], df["HM_AVG"])
    df["HM_AVG"] = np.clip(df["HM_AVG"], 0, 100)

# (C) PV_AVG
if "PV_AVG" in df.columns:
    def es_hpa(T): return 6.112 * np.exp(17.67 * T / (T + 243.5))
    can_calc = df["PV_AVG"].isna() & df.get("TA_AVG").notna() & df.get("HM_AVG").notna()
    if can_calc.any():
        df.loc[can_calc, "PV_AVG"] = (df.loc[can_calc, "HM_AVG"] * es_hpa(df.loc[can_calc, "TA_AVG"]) / 100.0)
    miss = df["PV_AVG"].isna()
    if miss.any(): 
        df.loc[miss, "PV_AVG"] = doy_stat_past(df["PV_AVG"], df["date"], "median")[miss]
    df.loc[df["PV_AVG"] < 0, "PV_AVG"] = np.nan

# (D) EV_*
if "EV_S" in df.columns:
    df = fill_short_ffill(df, "EV_S", 2)
    miss = df["EV_S"].isna()
    if miss.any(): 
        df.loc[miss, "EV_S"] = doy_stat_past(df["EV_S"], df["date"], "median")[miss]
if "EV_L" in df.columns:
    df = fill_short_ffill(df, "EV_L", 2)
    if {"EV_S","EV_L"}.issubset(df.columns):
        with np.errstate(divide='ignore', invalid='ignore'):
            ratio_ev = (df["EV_L"]/df["EV_S"]).replace([np.inf, -np.inf], np.nan).rolling(60, min_periods=15).median()
        miss = df["EV_L"].isna() & df["EV_S"].notna()
        if miss.any(): 
            df.loc[miss, "EV_L"] = df.loc[miss, "EV_S"] * ratio_ev[miss]
    miss = df["EV_L"].isna()
    if miss.any(): 
        df.loc[miss, "EV_L"] = doy_stat_past(df["EV_L"], df["date"], "median")[miss]
for c in ["EV_S","EV_L"]:
    if c in df.columns: 
        df.loc[df[c] < 0, c] = np.nan

# (E) SS_*, SI_DAY
for c in [x for x in ["SS_DAY","SS_DUR","SI_DAY"] if x in df.columns]:
    df = fill_short_ffill(df, c, 2)
    miss = df[c].isna()
    if miss.any(): 
        df.loc[miss, c] = doy_stat_past(df[c], df["date"], "median")[miss]
if "SS_DAY" in df.columns: df["SS_DAY"] = df["SS_DAY"].clip(lower=0)
if "SS_DUR" in df.columns: df["SS_DUR"] = df["SS_DUR"].clip(lower=0)
if {"SS_DAY","SS_DUR"}.issubset(df.columns):
    df["SS_DAY"] = np.minimum(df["SS_DAY"], df["SS_DUR"])

# (F) WS_*
for c in [x for x in ["WS_AVG","WS_MAX"] if x in df.columns]:
    df = fill_short_ffill(df, c, 3)
if {"WS_AVG","WS_MAX"}.issubset(df.columns):
    with np.errstate(divide='ignore', invalid='ignore'):
        ratio = (df["WS_MAX"]/df["WS_AVG"]).replace([np.inf, -np.inf], np.nan).rolling(60, min_periods=15).median()
    miss = df["WS_MAX"].isna() & df["WS_AVG"].notna()
    if miss.any(): 
        df.loc[miss, "WS_MAX"] = df.loc[miss, "WS_AVG"] * ratio[miss]
    still = df["WS_AVG"].isna() & df["WS_MAX"].notna()
    if still.any(): 
        df.loc[still, "WS_AVG"] = df.loc[still, "WS_MAX"] / ratio[still]
for c in [x for x in ["WS_AVG","WS_MAX"] if x in df.columns]:
    miss = df[c].isna()
    if miss.any(): 
        df.loc[miss, c] = doy_stat_past(df[c], df["date"], "median")[miss]
    df[c] = df[c].clip(lower=0)
if {"WS_AVG","WS_MAX"}.issubset(df.columns):
    df["WS_MAX"] = np.maximum(df["WS_MAX"], df["WS_AVG"])

# ===== 4) 윈저라이즈(선택, 기후만) =====
if APPLY_WINSOR:
    for c in [v for v in CLIMATE_VARS if v in df.columns]:
        df[c] = winsorize_by_doy_past(df, c, df["date"])

# ===== 5) 가격(2018–2024) 결측 보간 =====
if price_col in df.columns:
    s = df[price_col].copy()
    s[s < 0] = np.nan
    mask = df["year"].between(2018, 2024)
    s_sub = s.where(mask, np.nan)
    rl = run_lengths_na(s_sub)
    short = (rl > 0) & (rl <= PRICE_SHORT_GAP_MAX)
    s_sub[short] = s_sub.ffill()[short]
    roll_med = s_sub.rolling(PRICE_ROLL, min_periods=max(2, PRICE_ROLL//2)).median()
    need = mask & s_sub.isna(); s_sub[need] = roll_med[need]
    need = mask & s_sub.isna()
    if need.any():
        fill = doy_stat_past(s_sub, df["date"], "median")
        s_sub[need] = fill[need]
    if PRICE_IMPUTE_MODE == "two_sided":
        need = mask & s_sub.isna()
        if need.any():
            center_med = s_sub.rolling(PRICE_ROLL, min_periods=2, center=True).median()
            s_sub[need] = center_med[need]
    s[mask] = s_sub[mask]
    df[price_col] = s

# ===== 6) 최종: 전체-NaN 칼럼 드롭(보호: date/year/price) =====
protected = {"date", "year"}
if price_col in df.columns: 
    protected.add(price_col)
drop_cols = [c for c in df.columns if (c not in protected) and df[c].isna().all()]
if drop_cols:
    print("드롭(최종 전체-NaN):", drop_cols)
    df = df.drop(columns=drop_cols)

# ===== 저장 =====
keep = ["date","year"] + [c for c in CLIMATE_VARS if c in df.columns] + ([price_col] if price_col in df.columns else [])
keep = [c for c in keep if c in df.columns and not df[c].isna().all()]
df_clean = df[keep].copy()
df_clean.to_csv(FILE_OUT, index=False)

# ===== 로그 =====
print("저장 완료:", FILE_OUT)
print("기간:", df_clean["date"].min().date(), "→", df_clean["date"].max().date())
if price_col in df_clean.columns:
    m2017 = df_clean.loc[df_clean["year"]==2017, price_col].isna().sum()
    m1824 = df_clean.loc[df_clean["year"].between(2018, 2024), price_col].isna().sum()
    print(f"2017 price 결측(유지): {m2017} | 2018–2024 price 결측(보간 후): {m1824}")
print("남은 전체-NaN 칼럼 수:", int((df_clean.isna().all()).sum()))

드롭(원본 전체-NaN): ['EV_S', 'EV_L', 'SI_DAY', 'RN_DUR', 'SD_NEW', 'SD_MAX']
저장 완료: 오이_preprocessed.csv
기간: 2017-01-01 → 2024-12-31
2017 price 결측(유지): 365 | 2018–2024 price 결측(보간 후): 0
남은 전체-NaN 칼럼 수: 0


In [36]:
df_clean.head(20)  # 상위 20행

,date,year,TA_AVG,TA_MAX,TA_MIN,TD_AVG,TS_AVG,TG_MIN,HM_AVG,HM_MIN,PV_AVG,SS_DAY,SS_DUR,RN_DAY,RN_60M_MAX,WS_AVG,WS_MAX,PRICE
0,2017-01-01,2017,2.3,8.7,-2.1,-0.4,1.6,-6.4,83.1,59.0,6.0,5.2,9.7,0.0,0.0,0.6,2.3,NaN
1,2017-01-02,2017,4.8,11.0,-0.3,1.4,3.5,-4.8,79.5,60.0,6.8,3.3,9.7,0.0,0.0,1.2,3.2,NaN
2,2017-01-03,2017,2.1,8.8,-3.1,-2.6,2.4,-7.6,74.3,33.0,5.1,7.2,9.7,0.0,0.0,1.0,3.5,NaN
3,2017-01-04,2017,2.8,10.0,-2.7,-3.3,2.4,-6.9,68.9,26.0,4.9,5.9,9.7,0.0,0.0,0.9,3.7,NaN
4,2017-01-05,2017,3.9,9.0,-2.5,-2.0,2.1,-5.8,67.0,50.0,5.3,0.2,9.7,0.0,0.0,1.9,5.4,NaN
5,2017-01-06,2017,4.8,10.8,-0.2,-0.9,3.8,-4.7,68.3,44.0,5.7,6.3,9.8,0.0,0.0,1.4,2.9,NaN
6,2017-01-07,2017,2.4,9.3,-3.6,-1.2,2.2,-7.3,78.8,51.0,5.6,5.4,9.8,0.0,0.0,0.4,1.9,NaN
7,2017-01-08,2017,6.9,12.4,1.2,0.6,5.1,-3.3,66.1,44.0,6.4,5.0,9.8,0.0,0.0,1.4,3.5,NaN
8,2017-01-09,2017,3.0,6.3,-1.8,-2.4,4.1,-7.8,68.5,49.0,5.2,5.4,9.8,0.0,0.0,2.2,4.9,NaN
9,2017-01-10,2017,-1.2,2.5,-4.6,-9.1,-0.3,-8.8,55.8,41.0,3.2,6.3,9.8,0.0,0.0,1.7,4.1,NaN


In [20]:
# 고구마만
df_clean.drop(columns=["SI_DAY"], inplace=True)
df_clean.to_csv("고구마_preprocessed.csv", index=False)